In [ ]:
import csv
import re
import numpy as np
import json
from collections import defaultdict


file_path = "/home/atuin/g102ea/shared/group_10/datasets/charades/anotations/Charades/Charades_v1_test.csv"  

actions_info_dict = defaultdict(lambda: {"number": 0, "ratio": 0, "video_ids": []})

total_actions = 0


all_videos = set()  # save all video info

with open(file_path, mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)

    for row in reader:
        actions_str = row['actions'] 
        video_id = row['id']  
        all_videos.add(video_id)  
        action_codes = re.findall(r'c\d+', actions_str)  # match actions index
        
        # update actions_info_dict
        for action in action_codes:
            actions_info_dict[action]["number"] += 1
            total_actions += 1  # count the number of actions appealing
            if video_id not in actions_info_dict[action]["video_ids"]:
                actions_info_dict[action]["video_ids"].append(video_id)


for action, info in actions_info_dict.items():
    info["ratio"] = info["number"] / total_actions if total_actions > 0 else 0.0

# sort
actions_info = dict(sorted(actions_info_dict.items(), key=lambda x: x[1]["number"]))

In [17]:
N_TOTAL = 1500

In [18]:
n_samples_per_action = {k: int(np.ceil(v["ratio"] * N_TOTAL)) for k, v in actions_info.items()}

In [ ]:
from dataset.charades_mcq import parse_video_action_csv, parse_action_id_mapping
labels = parse_video_action_csv("/home/atuin/g102ea/shared/group_10/datasets/charades/anotations/Charades/Charades_v1_test.csv")

In [19]:
video_actiontime = {i["id"]: {"actions": i["actions"], "length": i["length"]} for i in labels}

In [69]:
localization = {i: {j[0]: min(1, (min(float(v["length"]), float(j[2])) - max(float(j[1]), 0))/float(v["length"])) for j in v["actions"]} for i, v in video_actiontime.items()}

In [83]:
final_set = {}
for action in n_samples_per_action:
    n_samples_action = n_samples_per_action[action]
    videos_for_action = actions_info[action]
    available_localizations = sorted([(localization[i][action], i) for i in videos_for_action["video_ids"]])
    final_set[action] = {"number": n_samples_action,
                        "ratio": videos_for_action["ratio"],
                        "videos": [i[1] for i in available_localizations][:n_samples_action]}

In [97]:
json.dump(final_set, open("/home/atuin/g102ea/shared/group_10/datasets/charades/subset_charades_hard.json", "w"))